### Importing packages

In [37]:
import ansys.fluent.core as pyfluent
meshing_session = pyfluent.launch_fluent(precision="double", processor_count=4, mode="meshing", show_gui=True)
workflow = meshing_session.workflow
meshing = meshing_session.meshing

### Initializing workflow for meshing

In [38]:
workflow.InitializeWorkflow(WorkflowType=r'Watertight Geometry')
meshing.GlobalSettings.LengthUnit.set_state(r'um')
meshing.GlobalSettings.AreaUnit.set_state(r'um^2')
meshing.GlobalSettings.VolumeUnit.set_state(r'um^3')

### Import geometry

Replace file_location with the actual address of the Static Mixer geometry file.
###### Eg: workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileName': r'D:/PyFluent/Static Mixer geometry.dsco',})


In [39]:
workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileName': r'D:\Work\Summer Placement Testbed\automationtest.scdoc',
                                                             "LengthUnit": "um",})
workflow.TaskObject['Import Geometry'].Execute() 

True

### Add local sizing

In [40]:
workflow.TaskObject['Add Local Sizing'].AddChildAndUpdate()

False

### Generate the Surface Mesh

In [41]:
workflow.TaskObject['Generate the Surface Mesh'].Execute()

True

### Describe geometry

In [42]:
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=False)
workflow.TaskObject['Describe Geometry'].Arguments.set_state({r'SetupType': r'The geometry consists of only fluid regions with no voids',})
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=True)
workflow.TaskObject['Describe Geometry'].Execute()

True

### Updating boundaries

In [43]:
workflow.TaskObject['Update Boundaries'].Execute()

True

### Updating regions

In [44]:
workflow.TaskObject['Update Regions'].Execute()

True

### Add boundary layers

In [45]:
workflow.TaskObject['Add Boundary Layers'].Arguments.set_state({r'LocalPrismPreferences': {r'Continuous': r'Stair Step',},})
workflow.TaskObject['Add Boundary Layers'].AddChildAndUpdate()

True

### Generate the volume Mesh

In [46]:
workflow.TaskObject["Generate the Volume Mesh"].Arguments = {"VolumeFill": "poly-hexcore",}

workflow.TaskObject['Generate the Volume Mesh'].Execute()

True

### Switch to solution

In [47]:
solver = meshing_session.switch_to_solver()

### Enabling energy

In [48]:
#solver.setup.models.energy = {"enabled" : False}

### Copying Water_liquid to materials panel

In [49]:
#solver.execute_tui(r'''/define/materials/copy fluid water-liquid ''')

### Assigning water_liquid to fluid domain

In [50]:
#solver.setup.cell_zone_conditions.fluid['fluid'] = {"material" : "water-liquid"}

### Assigning boudary conditons

In [51]:
solver.setup.boundary_conditions.velocity_inlet['inlet'].vmag = 3.78

### Residuals

In [52]:
solver.tui.solve.monitors.residual.plot("yes")

### Create surface report definition

In [53]:
solver.tui.solve.report_definitions.add(
    "outlet",
    "surface-massavg",
    "field",
    "velocity-magnitude",
    "surface-names",
    "outlet",
    "()",
    "quit",
)

### Hybrid Initialization

In [54]:
solver.solution.initialization.hybrid_initialize()

### Setting 20 iterations

In [55]:
solver.solution.run_calculation.iterate(iter_count = 5)

### Create definition for velocity magnitude contours
Create and display a definition for the velocity magnitude contours on the
symmetry plane.

- Set ``"contour"`` to ``"contour-vel"``.
- Set ``"field"`` to ``"velocity-magnitude"``.
- Set ``"surfaces-list"`` to ``"symmetry-xyplane"``.
- Set ``"display"`` to ``"contour-vel contour"``.



In [58]:
solver.tui.display.objects.create(
    "contour",
    "contour-vel",
    "filled?",
    "yes",
    "node-values?",
    "yes",
    "field",
    "velocity-magnitude",
    "surfaces-list",
    "symmetry-xyplane",
    "()",
    "coloring",
    "banded",
    "quit",
)

In [70]:
solver.tui.surface.plane(
    "yz-contour-plane",
    "yz-plane",
    "0"
)

## Create velocity vectors
Create and display velocity vectors on the symmetry-xyplane plane.

- Set ``"vector"`` to ``"vector-vel"``.
- Set ``"style"`` to ``"arrow"``.
- Set ``"surface-list"`` to ``"symmetry-xyplane"``.
- Set ``"scale"`` to ``"4"``.
- Set ``"skip"`` to ``"2"``.



In [59]:
solver.tui.display.objects.create(
    "vector",
    "vector-vel",
    "style",
    "arrow",
    "surface-list",
    "yz-contour-plane",
    "()",
    "scale",
    "scale-f",
    "4",
    "quit",
    "skip",
    "2",
    "quit",
)

## Display and save XY plot
Display and save an XY plot of the temperature profile across the centerline
of the outlet for the initial solution.



In [ ]:
solver.tui.display.objects.create(
    "xy",
    "xy-outlet-temp",
    "y-axis-function",
    "temperature",
    "surfaces-list",
    "z=0_outlet",
    "()",
    "quit",
)

### Exiting solver

In [ ]:
#solver.exit()